In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Data processing**

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import itertools


#  movies_df


In [ ]:
# movies_df = pd.read_csv('/content/drive/MyDrive/Colab/miniMovies.csv')
movies_df = pd.read_csv('/content/drive/MyDrive/Colab/movies.dat', sep="::", names=["movieId", "title", "genres"])

movies_df

<ipython-input-41-a1bbc6dd2c48>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_df = pd.read_csv('/content/drive/MyDrive/Colab/movies.dat', sep="::", names=["movieId", "title", "genres"])


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
#Generate "year" column and turn "genres" into a list
# movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
# movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
# movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
# movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df = movies_df.drop(columns = 'title')
movies_df = movies_df.set_index('movieId')
# movies_df.astype({'year': 'int16'})
movies_df

,genres
movieId,
1,"[Animation, Children's, Comedy]"
2,"[Adventure, Children's, Fantasy]"
3,"[Comedy, Romance]"
4,"[Comedy, Drama]"
5,[Comedy]
...,...
3948,[Comedy]
3949,[Drama]
3950,[Drama]


In [ ]:
#Isolate each genre from "genres"
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        movies_df.at[index, genre] = 1

movies_df = movies_df.fillna(0)
movies_df = movies_df.drop(columns = ['genres'])
movies_df = movies_df.astype('float16')
movies_df

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
movieId,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3950,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# ratings_df

In [ ]:
# ratings_df = pd.read_csv('/content/drive/MyDrive/Colab/miniRating.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/Colab/ratings.dat', sep = "::", names = ["userId", "movieId", "rating", "timestamp"], dtype = {'userId':'int32', 'movieId':'int32', 'rating':'float16'}) 
ratings_df

<ipython-input-44-11b87df5fbe9>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_df = pd.read_csv('/content/drive/MyDrive/Colab/ratings.dat', sep = "::", names = ["userId", "movieId", "rating", "timestamp"], dtype = {'userId':'int32', 'movieId':'int32', 'rating':'float16'})


,userId,movieId,rating,timestamp
0,1,1193,5.0,978300760
1,1,661,3.0,978302109
2,1,914,3.0,978301968
3,1,3408,4.0,978300275
4,1,2355,5.0,978824291
...,...,...,...,...
1000204,6040,1091,1.0,956716541
1000205,6040,1094,5.0,956704887
1000206,6040,562,5.0,956704746
1000207,6040,1096,4.0,956715648


Split train - test set (10%)

In [ ]:
from sklearn.model_selection import train_test_split
r_train, r_test = train_test_split(ratings_df, test_size=0.1, stratify=ratings_df['userId'])

Item-user matrix on training set

In [ ]:
movieId = movies_df.index
userId = ratings_df['userId'].drop_duplicates()
originalRating = np.full([max(movieId), max(userId)], np.nan)


for line in r_train.itertuples():
    # user_id = line[1][0]
    # movie_id = line[1][1]
    # rating = line[1][2]
    # originalRating[movie_id][user_id] = rating
    # originalRating[line[1][1]][line[1][0]] = line[1][2]
    originalRating[line[2]-1][line[1]-1] = line[3]



originalRating




array([[ 5., nan, nan, ..., nan, nan,  3.],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

# **Split Training - Test set**

In [ ]:
# a = sorted(ratingUseri.index)
# random.shuffle(a)
# print(random.shuffle([0, 1, 2, 3]))
# print(a)
# (sorted(ratingUseri.index))

In [ ]:
# import random
# trainRating = []
# testRating = []
# userId = ratings_df['userId'].drop_duplicates().tolist()
# for i in userId:
#     ratingUseri = ratings_df.loc[ratings_df['userId'] == i]
#     index = ratings_df.index
#     split = int(len(ratingUseri.index)*0.8)
#     temp = sorted(ratingUseri.index)
#     random.shuffle(temp)
#     trainRating.append(ratingUseri.loc[temp[:split]])
#     testRating.append(ratingUseri.loc[temp[split:]])


In [ ]:
# trainRating = pd.concat(trainRating, ignore_index=True)
# trainRating

In [ ]:
# #Generate training dataframe
# ratings_dict = {}
# for line in trainRating.iterrows():
#     user_id = line[1][0]
#     movie_id = line[1][1]
#     rating = line[1][2]

#     if movie_id not in ratings_dict:
#         ratings_dict[movie_id] = {}

#     ratings_dict[movie_id][user_id] = rating
# ratings_dict = dict(sorted(ratings_dict.items()))

# trainRating = pd.DataFrame(ratings_dict)
# trainRating = trainRating.T
# trainRating = trainRating.sort_index(axis=1)
# trainRating



In [ ]:
# testRating = pd.concat(testRating, ignore_index=True)
# testRating

In [ ]:
# #Generate test dataframe
# ratings_dict = {}
# for line in testRating.iterrows():
#     user_id = line[1][0]
#     movie_id = line[1][1]
#     rating = line[1][2]

#     if movie_id not in ratings_dict:
#         ratings_dict[movie_id] = {}

#     ratings_dict[movie_id][user_id] = rating
# ratings_dict = dict(sorted(ratings_dict.items()))

# testRating = pd.DataFrame(ratings_dict)
# testRating = testRating.T
# testRating = testRating.sort_index(axis=1)
# testRating



# **Content Based**

# ***User-User***


Normalize Item-user matrix

In [ ]:
# Normalize Item-user matrix
item_user = (originalRating-np.nanmean(originalRating, axis=0))
# item_user 


Get all movies are not watched by each person 

In [ ]:
# Get all users do not watched movie i
indexNanMovie = [0 for i in range(max(movieId))]
for i in range(max(movieId)):
    indexNanMovie[i] = np.where(np.isnan(item_user[i]))[0]
# indexNanMovie

Get all movies watched by each person

In [ ]:
# Get all movies watched by each person
indexRatedMovie = [0 for i in range(max(movieId))]
for i in range(max(movieId)):
    indexRatedMovie[i] = np.where(~np.isnan(item_user[i]))[0]
# indexRatedMovie


Fill nan value in Item-user matrix with 0

In [ ]:
# Fill nan value in Item-user matrix with 0
user_user = np.nan_to_num(item_user)
# user_user

Similarity func

In [ ]:
# Similarity func
import math
def cos(x,y):
    if x.dot(y)==0:
        return 0
    return x.dot(y)/math.sqrt(np.sum(x**2)*np.sum(y**2))
    

Calculate Item similiarity matrix

In [ ]:
# Calculate Item similiarity matrix
similiarityUser = np.full([max(userId), max(userId)], np.nan)
for i in range(max(userId)):
    for j in range(i, max(userId)):
        sim = cos(user_user[:,i], user_user[:,j])
        similiarityUser[i][j] = sim
        similiarityUser[j][i] = sim
# similiarityUser

Predict nan rating 

In [ ]:
# Predict nan rating 
# 13min
movieId = r_test['movieId'].drop_duplicates()
predict = item_user.copy()
for i in movieId-1:
    rate = (item_user[i][~np.isnan(item_user[i])]) # All ratings of movie i
    rateIndex = indexRatedMovie[i] # users watch movie i
    nul = indexNanMovie[i] # users do not watch movie i
    sim = similiarityUser[nul]
    for u in range(len(sim)):
        predict[i][nul[u]] = (sim[u][rateIndex] @ rate / sum(np.abs(sim[u][rateIndex])))
# predict

<ipython-input-69-af19a50110b1>:11: RuntimeWarning: invalid value encountered in double_scalars
  predict[i][nul[u]] = (sim[u][rateIndex] @ rate / sum(np.abs(sim[u][rateIndex])))


Denormalize predict matrix

In [ ]:
# Denormalize predict matrix
predictUser = (predict + np.nanmean(originalRating, axis=0))
predictUser

array([[5.        ,        nan,        nan, ...,        nan,        nan,
        3.        ],
       [       nan, 3.32022416, 3.71116017, ..., 3.88440405, 3.57854799,
        3.13414651],
       [       nan, 3.23560327, 3.6880803 , ..., 3.69584187, 3.49596968,
        3.03408848],
       ...,
       [       nan, 3.91090657, 4.09099456, ..., 3.81625121, 4.02275144,
        3.70377938],
       [       nan, 4.04390296, 3.9410013 , ..., 3.60521951, 4.16938982,
        3.93961356],
       [       nan, 3.83462091, 4.02820829, ..., 3.89922872, 4.07860049,
        3.69257373]])

# ***Item-Item***


Normalize Item-user matrix

In [ ]:
# Normalize Item-user matrix
itemRating = (originalRating.T-np.nanmean(originalRating, axis=1)).T
# itemRating 


<ipython-input-47-ef420703ed1b>:2: RuntimeWarning: Mean of empty slice
  itemRating = (originalRating.T-np.nanmean(originalRating, axis=1)).T


Get all movies are not watched by each person 

In [ ]:
# Get all movies are not watched by each person
indexNanMovie = [0 for i in range(max(userId))]
for i in range(max(userId)):
    indexNanMovie[i] = np.where(np.isnan(itemRating.T[i]))[0]


Get all movies watched by each person

In [ ]:
# Get all movies watched by each person
indexRatedMovie = [0 for i in range(max(userId))]
for i in range(max(userId)):
    indexRatedMovie[i] = np.where(~np.isnan(itemRating.T[i]))[0]



Fill nan value in Item-user matrix with 0

In [ ]:
# Fill nan value in Item-user matrix with 0
item_item = np.nan_to_num(itemRating)
# item_item

Similarity func

In [ ]:
# Similarity func
import math
def cos(x,y):
    if x.dot(y)==0:
        return 0
    return x.dot(y)/math.sqrt(np.sum(x**2)*np.sum(y**2))
    

Calculate Item similiarity matrix

In [ ]:
# Calculate Item similiarity matrix
similiarityItem = np.full([max(movieId), max(movieId)], np.nan)
for i in range(max(movieId)):
    for j in range(i, max(movieId)):
        sim = cos(item_item[i], item_item[j])
        similiarityItem[i][j] = sim
        similiarityItem[j][i] = sim
# similiarityItem

Predict nan rating 

In [ ]:
# Predict nan rating 
# 13min
predict = itemRating.copy()
userId = r_test['userId'].drop_duplicates()
for u in userId-1:
    # u = 4
    rate = (itemRating.T[u][~np.isnan(itemRating.T[u])]) # All ratings of u
    rateIndex = indexRatedMovie[u]
    nul = indexNanMovie[u] # movies are not watched by u
    sim = similiarityItem[nul]
    for i in range(len(sim)):
        predict[nul[i]][u] = sim[i][rateIndex] @ rate / sum(np.abs(sim[i][rateIndex]))
# predict

<ipython-input-53-6a0f9f7dd022>:12: RuntimeWarning: invalid value encountered in double_scalars
  predict[nul[i]][u] = sim[i][rateIndex] @ rate / sum(np.abs(sim[i][rateIndex]))


Denormalize predict matrix

In [ ]:
# Denormalize predict matrix
predictItem = (predict.T + np.nanmean(originalRating, axis=1)).T
predictItem

<ipython-input-54-3a157c2f4a6c>:2: RuntimeWarning: Mean of empty slice
  predictItem = (predict.T + np.nanmean(originalRating, axis=1)).T


array([[5.        , 4.18561815, 4.28441173, ..., 3.89434792, 4.09716891,
        3.        ],
       [3.56420349, 3.19764445, 3.58011036, ..., 3.64034922, 3.15266771,
        2.61003268],
       [3.55951061, 3.14508789, 3.42999765, ..., 2.62893748, 2.98901679,
        2.49395137],
       ...,
       [3.98978107, 3.73044147, 4.03790949, ..., 3.50226219, 3.73998065,
        3.54576375],
       [4.2061161 , 3.96985097, 3.90917769, ..., 4.09055947, 4.07518411,
        3.86660001],
       [4.05318195, 3.75550286, 3.96248483, ..., 4.24615536, 3.80387208,
        3.56355527]])

# **Matrix Factorization**


# **Evaluate accuracy**

In [ ]:
Item_se = 0
User_se = 0
for r in r_test.itertuples():
    if not np.isnan(predictItem[r[2]-1][r[1]-1]):
        Item_se+=(predictItem[r[2]-1][r[1]-1]-r[3])**2
        User_se+=(predictUser[r[2]-1][r[1]-1]-r[3])**2

print("Item_se: ", end="")
print(math.sqrt(Item_se/len(r_test)))
print("User_se: ", end="")
print(math.sqrt(User_se/len(r_test)))




Item_se: 0.8741139314098685
User_se: nan
